### Download Data 

* use coco dataset 2017 
* to save time, use validation dataset as training set 
* make folder structure to fulfill the bring your container folder structure 

In [ ]:
%%bash 
cd ~/SageMaker
mkdir input 
mkdir input/data 
cd input/data 
wget -O coco.zip https://tinyurl.com/yhtbxr6q 
unzip coco.zip 

### Build AlphaPose container 

* reference to [install instruction](https://github.com/catwhiskers/AlphaPose/blob/master/docs/INSTALL.md)
* fix package dependency pycocotools==2.0.2a1
* download pretrained models for inferences (yolov3 and fastpose)


In [2]:
%%bash 
cd container 
./build_and_push.sh 

230755935769
us-west-2
Login Succeeded
Login Succeeded
1.15.5-gpu-py37-cu100-ubuntu18.04: Pulling from tensorflow-training
Digest: sha256:b9bf85ad4689c7a728ba47866fdfe868ee7c7bcdb0417b3939df4cbc7053a5bc
Status: Image is up to date for 763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:1.15.5-gpu-py37-cu100-ubuntu18.04
763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:1.15.5-gpu-py37-cu100-ubuntu18.04

Step 1/4 : ARG BASE_IMG=${BASE_IMG}
Step 2/4 : FROM ${BASE_IMG}
 ---> 53fefee0e5ac
Step 3/4 : RUN apt-get update -y && apt-get install ffmpeg libsm6 libxext6 -y
 ---> Using cache
 ---> 5242af8b2332
Step 4/4 : RUN pip install nibabel opencv-python matplotlib keras==2.3.1
 ---> Using cache
 ---> a08698b5d41b
Successfully built a08698b5d41b
Successfully tagged dunet:latest
The push refers to repository [230755935769.dkr.ecr.us-west-2.amazonaws.com/dunet]
156f3779f1f9: Preparing
c62bd00454db: Preparing
85d8371500dd: Preparing
9130585b9ca3: Preparing
9036801cf049: 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [8]:
import boto3 
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
image_uri = "{}.dkr.ecr.{}.amazonaws.com/dunet".format(account_id, "us-west-2")
image_uri

'230755935769.dkr.ecr.us-west-2.amazonaws.com/dunet'

### Training locally based on the docker image 

In [16]:
!nvidia-docker run -it -v $PWD:/opt/ml 230755935769.dkr.ecr.us-west-2.amazonaws.com/dunet

Using TensorFlow backend.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2021-05-02 13:59:28.871037: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-02 13:59:29.045926: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-02 13:59:29.047667: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:17.0
2021-05-02 13:59:29.047778: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-02 13:59:29.049422: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 1 with pr

2021-05-02 13:59:29.464878: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-02 13:59:29.466553: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-02 13:59:29.468186: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-02 13:59:29.469775: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1325] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10812 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:17.0, compute capability: 3.7)
2021-05-02 13:59:29.470510: I tensorflow/stream_executor/cuda/cud

global_average_pooling2d_3 (Glo (None, 128)          0           conv2d_8[0][0]                   
__________________________________________________________________________________________________
global_average_pooling2d_4 (Glo (None, 128)          0           activation_12[0][0]              
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 1, 1, 128)    0           global_average_pooling2d_3[0][0] 
__________________________________________________________________________________________________
reshape_4 (Reshape)             (None, 1, 1, 128)    0           global_average_pooling2d_4[0][0] 
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 1, 1, 8)      1024        reshape_3[0][0]                  
__________________________________________________________________________________________________
dense_7 (D

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
no loading weight!
Stroke_segment.py:36: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  h5 = h5py.File('/opt/ml/input/data/atlas/h5/train')
['data_val', 'label_val']
Stroke_segment.py:41: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  h5 = h5py.File('/opt/ml/input/data/atlas/h5/test_0.8')
<HDF

### Use SageMaker Training Jobs 

In [15]:
import sagemaker
from sagemaker import get_execution_role
role = get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = "dunet"

#### upload data to s3 

In [17]:
!cd ./input/data/ && aws s3 cp --recursive atlas s3://{bucket}/{prefix}/atlas

upload: atlas/h5/test to s3://sagemaker-us-west-2-230755935769/dunet/atlas/h5/test
upload: atlas/h5/detection/test to s3://sagemaker-us-west-2-230755935769/dunet/atlas/h5/detection/test
upload: atlas/h5/test_0.8 to s3://sagemaker-us-west-2-230755935769/dunet/atlas/h5/test_0.8
upload: atlas/h5/train to s3://sagemaker-us-west-2-230755935769/dunet/atlas/h5/train
upload: atlas/h5/detection/train to s3://sagemaker-us-west-2-230755935769/dunet/atlas/h5/detection/train
upload: atlas/h5/train_0.8 to s3://sagemaker-us-west-2-230755935769/dunet/atlas/h5/train_0.8


#### define s3 input and output paths 

In [18]:
atlas_h5_data = "s3://{}/{}/atlas/".format(bucket, prefix)
outpath = "s3://{}/{}/output/".format(bucket, prefix)
repositoryUri = image_uri

#### define job_name and  and hyperparameters

In [19]:
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
job_name = "dunet-{}".format(str(int(timestamp))) 
job_name 

'dunet-1619964530'

#### submit training job 

In [20]:
atlas_input = sagemaker.inputs.TrainingInput(atlas_h5_data)

In [23]:
# from  sagemaker.tensorflow.estimator import TensorFlow

# from sagemaker.pytorch import PyTorch

estimator = sagemaker.estimator.Estimator( 
                        role=role,
                        image_uri=repositoryUri,
                        instance_count=1,
                        instance_type='ml.p3.8xlarge',
                        sagemaker_session=sagemaker_session,
                        volume_size=100, 
                        debugger_hook_config=False
                   )


In [ ]:
estimator.fit(inputs={"atlas":atlas_input}, job_name=job_name)

2021-05-02 14:14:06 Starting - Starting the training job...
2021-05-02 14:14:29 Starting - Launching requested ML instancesProfilerReport-1619964846: InProgress
.........
2021-05-02 14:16:02 Starting - Preparing the instances for training.........
2021-05-02 14:17:29 Downloading - Downloading input data....................................
2021-05-02 14:23:34 Training - Downloading the training image........